# Deploying your first machine learning model on Google Cloud Platform

This notebook complements the presentation slides and contains the little code required to deploy a ML model on GCP.

## Fitting and saving a model

In [1]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import joblib

iris = datasets.load_iris()
classifier = RandomForestClassifier()
classifier.fit(iris.data, iris.target)

joblib.dump(classifier, 'model.joblib')

['model.joblib']

## Save a random example which we will test on

In [2]:
import random
import json


with open('test_example.json', 'w') as f:
    ex = iris['data']
    ex = random.choice(ex)
    ex = ex.tolist()
    json.dump(ex, f)

## Install the GCP SDK 

[Guide here](https://cloud.google.com/sdk/docs/install)


## Deploying the model

In [11]:
! export PROJECT_ID=$(gcloud config list project --format "value(core.project)")
! export BUCKET_NAME=${PROJECT_ID}-aiplatform

In [12]:
! echo $BUCKET_NAME

In [5]:
! REGION=europe-west2

## Make the bucket (can also be done through console like everything else)

In [6]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

CommandException: The mb command requires at least 1 argument. Usage:

  gsutil mb [-b (on|off)] [-c <class>] [-l <location>] [-p <proj_id>]
            [--retention <time>] gs://<bucket_name>...

For additional help run:
  gsutil help mb


## Copy your model to the cloud

In [ ]:
! gsutil cp ./model.joblib gs://your-bucket/model.joblib

## Test a model locally

In [13]:
! gcloud ai-platform local predict --model-dir . --json-instances test_example.json --framework scikit-learn

If the signature defined in the model is not serving_default then you must specify it via --signature-name flag, otherwise the command may fail.
[2]


## Make prediction through cloud

In [14]:
! gcloud ai-platform predict --model test_model --json-instances test_example.json --region europe-west2

Using endpoint [https://europe-west2-ml.googleapis.com/]
[2]



## Python bindings

### Install

In [15]:
! pip install google-api-python-client

### Authenticate

In [16]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/ice/keys/gcp-my-first-project.json"

### Enter project and model details

In [17]:
project = 'fifth-marker-303416'
model = 'globalDemoClassifier'

### Get predictions

In [18]:
import json
import googleapiclient.discovery

ai_platform_client = googleapiclient.discovery.build('ml', 'v1')

with open('test_example.json') as f:
    instances = json.load(f)

name = f'projects/{project}/models/{model}'

response = ai_platform_client.projects().predict(
    name=name,
    body={'instances': [instances]}
).execute()

print(response)

{'predictions': [2]}
